# A1.4 Selección de características

1. Importar los datos del archivo **Vino Tinto.csv** al ambiente de trabajo. Se revisan las dimensiones del dataframe y los datos ingresados.

In [18]:
import pandas as pd

# Se crea el DataFrame con los datos del archivo
df = pd.read_csv('A1.4 Vino Tinto.csv')

# Se imprimen las dimensiones del DataFrame
print(f"Dimensiones del DataFrame (filas, columnas): {df.shape}")

# Se imprimen las primeras 5 filas de datos
print("\nPrimeras 5 filas de datos\n")
df.head(5)

Dimensiones del DataFrame (filas, columnas): (1599, 12)

Primeras 5 filas de datos



,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


2. Separar el DataFrame en datos de entrenamiento y datos de prueba con una proporción 80/20, asegurando que la partición sea aleatoria. Se imprime en pantalla las dimensiones de ambos conjuntos de datos, comprobando que la cantidad de observaciones de ambos conjuntos de datos sumen a la cantidad de datos original.

In [20]:
from sklearn.model_selection import train_test_split

# Se generan los datos de entrenamiento (80%) y validación (20%)
train, test = train_test_split(df, train_size = 0.8, random_state=42)

# Generar X e Y para entrenamiento
X_train = train.drop('calidad', axis = 1)
Y_train = train['calidad']

# Generar X e Y para prueba
X_test = test.drop('calidad', axis = 1)
Y_test = test['calidad']

# Se imprimen las dimensiones de ambos conjuntos
print(f"Dimensiones del conjunto de entrenamiento (filas, columnas): {X_train.shape}")
print(f"Dimensiones del conjunto de pruebas (filas, columnas): {X_test.shape}")

# Verificar si la suma de ambos conjuntos equivale al tamaño del DataFrame original
if train.shape[0] + test.shape[0] == df.shape[0]:
    print("Las dimensiones de los subconjuntos si pertenecen al conjunto original")
else:
    print("Las dimensiones de los subconjuntos no cuadran con las del conjunto original")

Dimensiones del conjunto de entrenamiento (filas, columnas): (1279, 11)
Dimensiones del conjunto de pruebas (filas, columnas): (320, 11)
Las dimensiones de los subconjuntos si pertenecen al conjunto original


3. Generar la metodología de selección hacia adelante imprimiendo en consola los índices o los nombres de las características seleccionadas. Se hace uso de la función **SequentialFeatureSelector** de la librería **mlxtend.feature_selection**.

In [30]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Se declara el modelo y se hace la metodología de selección hacia adelante con la función SFS. 
model = LinearRegression()
sfs_forward = SFS(model, k_features=(2,8), forward=True, floating=False, scoring='r2', cv=10)
sfs_forward.fit(X_train, Y_train)

# Se imprimen las características seleccionadas
select_features = list(sfs_forward.k_feature_names_)
print("Características seleccionadas: ", select_features)

Características seleccionadas:  ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


4. Entrenar un modelo que solamente contenga las variables seleccionadas, predecir la
respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo
usando la R cuadrada, imprimiendo dicho valor en consola.

In [35]:
from sklearn.metrics import r2_score

# Se obtienen los datos de solo las características previamente seleccionadas
X_train_sel = X_train[select_features]
X_test_sel = X_test[select_features]

# Se entrena el modelo con el nuevo dataset
model.fit(X_train_sel, Y_train)

# Se predice la respuesta en las observaciones de prueba (nuevo dataset)
y_pred = model.predict(X_test_sel)

# Se mide la capacidad de predicción a través de R2
r2_forward = r2_score(Y_test, y_pred)
print(f"R cuadrada con selección hacia adelante: {r2_forward}")

R cuadrada con selección hacia adelante: 0.40126288354402984


5. Realizar un proceso de selección hacia atrás a partir de las variables seleccionadas por
el método de selección hacia adelante e imprimir en consola los índices o nombres de las
variables seleccionadas. 

In [40]:
# Se hace la metodología de selección hacia atrás con la función SFS.
sfs_back = SFS(model, k_features=(2,5), forward=False, floating=False, scoring='r2', cv=10)
sfs_back.fit(X_train_sel, Y_train)

# Se listan los nombres de las características seleccionadas hacia atrás y se imprimen
selected_features_back = list(sfs_back.k_feature_names_)
print("Características seleccionadas hacia atrás: ", selected_features_back)

Características seleccionadas hacia atrás:  ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


6. Repetir el paso 4, pero para un modelo que contenga solamente las variables
seleccionadas en el paso 5. Se imprime en pantalla un breve texto que describe mi opinión
sobre la diferencia en R cuadrada medida entre los modelos de los pasos 4 y 6.

In [45]:
# Se obtienen los datos de solo las características previamente seleccionadas
X_train_sel_back = X_train[selected_features_back]
X_test_sel_back = X_test[selected_features_back]

# Se entrena el modelo con el nuevo dataset
model.fit(X_train_sel_back, Y_train)

# Se predice la respuesta en las observaciones de prueba (nuevo dataset)
y_pred_back = model.predict(X_test_sel_back)

# Se mide la capacidad de predicción a través de R2
r2_back = r2_score(Y_test, y_pred_back)
print(f"R Cuadrada con selección hacia atrás: {r2_back}")

# Se comparan los modelos para saber cual es el mejor
if r2_forward > r2_back:
    print(f"El modelo con selección hacia adelante es mejor, R2: {r2_forward}")
else:
    print(f"El modelo con selección hacia atrás es mejor, R2: {r2_back}")

R Cuadrada con selección hacia atrás: 0.3958889666765403
El modelo con selección hacia adelante es mejor, R2: 0.40126288354402984


En este caso, el modelo obtenido por el método de selección hacia adelante obtuvo una mejor R2 (0.401 > 0.395) que el obtenido por el método de selección hacia atrás. Aunque la diferencia entre ambas es mínima, el primer modelo se ajusta mejor que el segundo y por esa razón considero que es mejor.  